In [20]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [21]:
# Load the dataset (using your provided data as an example)
df = pd.read_csv("F:\ML\Wine-Quality-Prediction\data\wine_quality_processed.csv")
X = df.drop(columns=['quality']).values
y = df['quality'].values

# Convert quality to categorical
encoder = OneHotEncoder(sparse_output=False)  # Updated parameter name
y_categorical = encoder.fit_transform(y.reshape(-1, 1))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Check shapes
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")


X_train shape: (2318, 11), y_train shape: (2318, 6)


In [22]:
# Build the classification neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')  # Output layer for classification
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Loss: 0.4675, Test Accuracy: 0.8517


In [23]:
def activation_maximization_classification(target_class, model, iterations=500, learning_rate=0.01):
    # Start with a random input vector
    input_vector = tf.Variable(np.random.normal(size=(1, X_train.shape[1])), dtype=tf.float32)

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Gradient ascent loop
    for i in range(iterations):
        with tf.GradientTape() as tape:
            # Predict probabilities for the input vector
            prediction = model(input_vector)
            # Loss is the negative log probability of the target class
            loss = -tf.math.log(prediction[0, target_class] + 1e-8)

        # Compute gradients and update input vector
        grads = tape.gradient(loss, input_vector)
        optimizer.apply_gradients([(grads, input_vector)])

        # Clip values to ensure realistic inputs
        input_vector.assign(tf.clip_by_value(input_vector, -3, 3))

    return input_vector.numpy().flatten()

In [24]:
# unique values in the 'quality' column
unique_qualities = sorted(df['quality'].unique())
for quality in unique_qualities:
    target_class = encoder.categories_[0].tolist().index(quality)  # Index of class '8'
    optimized_input = activation_maximization_classification(target_class, model)
    print(f"Optimized Input for Class {target_class} (Quality {quality}): {optimized_input}")

Optimized Input for Class 0 (Quality 3): [ 0.38748193  0.04999524  0.49698764 -0.4817729   1.3638119  -0.9415769
 -1.0848961  -0.4249435   0.30951044  0.754299   -1.2895449 ]
Optimized Input for Class 1 (Quality 4): [ 0.10530781  1.1029971   1.2180761  -0.9723002  -0.7218239   0.37776414
  0.87326163 -1.6168528  -0.17493287  1.3660377  -1.6912322 ]
Optimized Input for Class 2 (Quality 5): [-2.8948002  -1.2893075   2.3054168   0.07964412 -0.6151767  -0.15165144
 -0.6584979   0.01741432 -0.23442198  1.0971041  -2.039202  ]
Optimized Input for Class 3 (Quality 6): [-1.6629272   0.66497386 -0.08048213  1.39567     2.7370055  -0.47814214
  0.50294423 -1.2171702  -1.1457922   0.09598751 -0.06951897]
Optimized Input for Class 4 (Quality 7): [ 1.6053342   0.9350312  -0.82308364  1.6216229  -0.8078967  -1.7058928
 -1.2533605  -0.27629316  0.47818202  0.6306382  -0.40634376]
Optimized Input for Class 5 (Quality 8): [ 0.36506522 -0.23210868  1.1577822   0.00278777 -0.7656591   0.7262541
 -0.79746

In [25]:
def counterfactual_explanation_classification(input_instance, target_class, model, iterations=500, learning_rate=0.01):
    # Convert the instance into a tensor
    input_tensor = tf.Variable(input_instance.reshape(1, -1), dtype=tf.float32)

    # Define the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    # Gradient descent loop
    for i in range(iterations):
        with tf.GradientTape() as tape:
            # Predict probabilities
            prediction = model(input_tensor)
            # Loss is the negative log probability of the target class + minimal changes
            loss = -tf.math.log(prediction[0, target_class] + 1e-8) + 0.01 * tf.reduce_sum(tf.square(input_tensor - input_instance))

        # Compute gradients and update input tensor
        grads = tape.gradient(loss, input_tensor)
        optimizer.apply_gradients([(grads, input_tensor)])

        # Clip the input values to realistic ranges
        input_tensor.assign(tf.clip_by_value(input_tensor, -3, 3))

    return input_tensor.numpy().flatten()


In [26]:
# Perform Counterfactual Explanation
index = 10
instance = X_test[index]
true_class = np.argmax(y_test[index])
target_quality = 8  # Class '8' is the best quality
target_class = encoder.categories_[0].tolist().index(target_quality)  # Index of class '8'

# Get the modified instance
modified_instance = counterfactual_explanation_classification(instance, target_class, model)

# Attribute names
attribute_names = [
    "Fixed Acidity", "Volatile Acidity", "Citric Acid", "Residual Sugar", 
    "Chlorides", "Free Sulfur Dioxide", "Total Sulfur Dioxide", 
    "Density", "pH", "Sulphates", "Alcohol"
]

# Print original and modified instances
print(f"Original Instance: {instance}")
print(f"True Class: {true_class}")
print(f"Modified Instance for Class {target_class} (Quality {target_quality}): {modified_instance}")

# Compute percentage change for each attribute
percentage_changes = ((modified_instance - instance) / instance) * 100

# Print the percentage changes with attribute names
print("\nPercentage Change in Each Attribute:")
for i, (attr_name, original, modified, pct_change) in enumerate(zip(attribute_names, instance, modified_instance, percentage_changes)):
    print(f"{attr_name}: Original={original:.4f}, Modified={modified:.4f}, Change={pct_change:.2f}%")


Original Instance: [ 1.59654248 -0.45300195  2.34810035  0.49275868 -0.44305005 -0.93846228
 -0.72982342  0.97160621 -0.90050392  0.01345795  0.70063152]
True Class: 3
Modified Instance for Class 5 (Quality 8): [ 1.6277124  -0.54907656  2.2424533   0.58880264 -0.64397824 -0.9718691
 -0.7299668   0.9086992  -1.0455141   0.1073776   0.7543377 ]

Percentage Change in Each Attribute:
Fixed Acidity: Original=1.5965, Modified=1.6277, Change=1.95%
Volatile Acidity: Original=-0.4530, Modified=-0.5491, Change=21.21%
Citric Acid: Original=2.3481, Modified=2.2425, Change=-4.50%
Residual Sugar: Original=0.4928, Modified=0.5888, Change=19.49%
Chlorides: Original=-0.4431, Modified=-0.6440, Change=45.35%
Free Sulfur Dioxide: Original=-0.9385, Modified=-0.9719, Change=3.56%
Total Sulfur Dioxide: Original=-0.7298, Modified=-0.7300, Change=0.02%
Density: Original=0.9716, Modified=0.9087, Change=-6.47%
pH: Original=-0.9005, Modified=-1.0455, Change=16.10%
Sulphates: Original=0.0135, Modified=0.1074, Chan